# wps_bccaq
A process that runs the [bccaq.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/BCCAQ.R) function from ClimDown.

Bias Correction/Constructed Analogues with Quantile mapping reordering (BCCAQ) is composed of the following steps.
- Constructed Analogues (CA)
- Climate Imprint (CI)
- Quantile Delta Mapping (QDM)
- Rerank


In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
url = 'http://127.0.0.1:5004/'
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [3]:
# NBVAL_IGNORE_OUTPUT
chickadee.bccaq?

Signature:
chickadee.bccaq(
    gcm_file,
    obs_file=None,
    var=None,
    end_date='2005-12-31',
    out_file='out.nc',
    loglevel='INFO',
)
Docstring:
Full statistical downscaling of coarse scale global climate model (GCM) output to a fine spatial resolution

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`
    GCM simulations in NetCDF format
obs_file : ComplexData:mimetype:`application/x-netcdf`
    high-res gridded historical observations
var : string
    Name of the NetCDF variable to downscale
end_date : string
    Defines the end of the calibration period
out_file : string
    Path to output file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    output netCDF file
File:      ~/code/birds/chickadee/</tmp/chickadee-venv/lib/python3.8/site-packages/birdy/client/base.py-1>
Type:      method


### Run the bccaq process

In [4]:
gcm_file = "tests/data/tiny_gcm.nc"
obs_file = "tests/data/tiny_obs.nc"
var = "tasmax"
end_date = "1972-12-31"

output = chickadee.bccaq(gcm_file, obs_file, var, end_date)
output.get()[0]

'http://localhost:5004/outputs/355a610a-1e13-11eb-bd25-87d68bc741c6/out.nc'

### Test for expected output

In [29]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))
    expected_data = Dataset('tests/data/bccaq_expected_output.nc')
    for key, value in expected_data.dimensions.items():
        assert str(output_data.dimensions[key]) == str(value)